# Estudo da segmentação de clientes

In [1]:
import pandas as pd

PATH_D = 'C:/Users/cpcle/OneDrive/Documentos/Celso/Python/veneta-dash/data/'

COMPRAS_PUBLICAS = ['FNDE', 'GM QUALITY COMERCIO LTDA',
                    'DEFENSORIA PUBLICA DA UNIAO']

GRANDES_LIVRARIAS = ['Livraria Travessa', 'Martins Fontes',
                     'G.D.A. Lorandi ME', 'Saraiva e Siciliano', 'Itiban', 'Bookwire Brazil Distribuidora de Livros Digitiais Ltda.', 'Livraria da Vila', 'Livraria Cultura']

def numero(valor):
    """Formata valor na notação brasileira.
    

    Parameters
    ----------
    valor : float
        Valor a ser formatado

    Returns
    -------
    str
        Valor formatado como #.##0,00
    """
    return '{:,.0f}'.format(valor).replace(',', '@#')\
        .replace('.', ',').replace('@#', '.')

# Lê arquivo do Excel (NFE's)        
df = pd.read_pickle(PATH_D + 'Notas.pkl')


In [2]:
# Lê tabelas de CFOP
ARQ = 'C:/Users/cpcle/OneDrive/Documentos/Celso/Veneta/Banco de Dados Faturamento.xlsm'
cad = pd.read_excel(ARQ, sheet_name='CFOP', header=0, usecols='A:B', dtype=str).dropna()

In [3]:
# Agrupa CFOP's em Venda, Consignação e Outro
grupos = {'Venda': ['1201', '1202', '2201', '2202',
                    '5101', '6101', '5102', '5104', '6102', '6108',
                    '5116', '6116', '5117', '6117', '5119', '6119'],
          'Consignação': ['5114', '6114', '9999']}

chaves = {}
for key in grupos.keys():
    for value in grupos[key]:
        chaves[value] = key

cad['Tipo'] = cad['CFOP'].map(chaves).fillna('Outro')

df = df.merge(cad, how='left', on='CFOP')
del cad


## Movimentos e seus respectivos Tipos

In [4]:
a = df.groupby(['CFOP', 'Descrição', 'Tipo'])['Receita Líquida']\
    .sum().sort_values(ascending= False).reset_index(level=[1, 2])

a['Receita Líquida'] = a['Receita Líquida'].map(numero)
a

,Descrição,Tipo,Receita Líquida
CFOP,,,
5102,Venda de mercadoria adquirida ou recebida de t...,Venda,9.053.108
6102,Venda de mercadoria adquirida ou recebida de t...,Venda,4.568.193
5114,Venda de mercadoria adquirida ou recebida de t...,Consignação,2.930.626
6114,Venda de mercadoria adquirida ou recebida de t...,Consignação,1.082.545
6108,Venda de mercadoria adquirida ou recebida de t...,Venda,488.276
5104,Venda de mercadoria adquirida ou recebida de t...,Venda,399.352
5116,Venda de produção do estabelecimento originada...,Venda,15.000
6119,Venda de mercadoria adquirida ou recebida de t...,Venda,6.114
6101,Venda de produção do estabelecimento,Venda,1.817


In [5]:
a = df.groupby(['Tipo'])['Receita Líquida']\
    .sum().sort_values(ascending=False).reset_index()
    
a['Receita Líquida'] = a['Receita Líquida'].map(numero)

a

,Tipo,Receita Líquida
0,Venda,13.892.215
1,Consignação,3.992.294
2,Outro,770


## Maiores Clientes desde Janeiro de 2020 - (Receita Líquida)

In [6]:
a = df.loc[(df['Emissao'] > '2020')].groupby(['Grupo', 'Tipo'], as_index=False).sum()

a = a.pivot_table(index=['Grupo'], columns=['Tipo'], values=['Receita Líquida'], fill_value=0)

a[('Receita Líquida', 'Total')] = a.sum(axis=1)

a = a['Receita Líquida'][['Venda', 'Consignação', 'Outro', 'Total']].sort_values(ascending=False, by=['Total'])

a.head(50).applymap(numero)

Tipo,Venda,Consignação,Outro,Total
Grupo,,,,
Amazon,3.050.282,0,0,3.050.282
FNDE,468.709,0,0,468.709
GM QUALITY COMERCIO LTDA,218.457,0,0,218.457
Inovação Distribuidora de Livros Ltda,148.387,30.753,0,179.140
Catavento Distribuidora de Livros S/A,77.974,39.617,0,117.590
Boa Viagem Distribuidora de Livros Ltda.,26.439,68.078,0,94.516
Eventos Veneta,94.401,0,0,94.401
A Página Distribuidora de Livros Ltda,59.424,30.614,0,90.038
Livraria Travessa,4.110,70.440,0,74.550


## Maiores Clientes (Trimestre a Trimestre) - (Receita Líquida)

In [7]:
a = df.copy()

filtro = ((a['CNPJ'].str.len() == 11) & (a['CNPJ'] != '01462931855')) | (
    a['Grupo'] == 'Eventos Veneta')

a.loc[filtro, 'Grupo'] = 'Pessoas Físicas'

a.loc[a['Grupo'].isin(COMPRAS_PUBLICAS), 'Grupo'] = 'Vendas Públicas'

a = a.loc[(a['Emissao'] > '2020') &
           (a['Tipo'].isin(['Venda', 'Consignação']))].groupby(
    ['Grupo', pd.Grouper(key='Emissao', freq='QS')])['Receita Líquida'].sum().sort_values(ascending=False).reset_index()

a = a.pivot_table(index='Grupo', columns='Emissao',
                  values='Receita Líquida', aggfunc=sum, fill_value=0)

a['Total'] = a.sum(axis=1)

a = a.sort_values(by='Total', ascending=False).head(50).applymap(numero)

b = ['{:d} - Q{:d}'.format(x.year, x.quarter) for x in a.columns[:-1]]
b.append(a.columns[-1])
a.columns = b

a.head(50)


,2020 - Q1,2020 - Q2,2020 - Q3,2020 - Q4,2021 - Q1,2021 - Q2,2021 - Q3,Total
Grupo,,,,,,,,
Amazon,141.741,322.429,651.686,1.142.687,112.213,313.870,365.658,3.050.282
Pessoas Físicas,118.556,25.716,68.929,331.736,46.505,242.254,44.102,877.797
Vendas Públicas,0,0,0,504.676,218.457,0,0,723.133
Inovação Distribuidora de Livros Ltda,2.918,5.076,7.287,51.782,27.274,62.120,22.683,179.140
Catavento Distribuidora de Livros S/A,866,1.965,11.794,52.501,15.666,16.763,18.037,117.590
Boa Viagem Distribuidora de Livros Ltda.,3.838,2.139,1.797,3.436,27.722,31.878,23.707,94.516
A Página Distribuidora de Livros Ltda,4.909,2.047,26.598,15.386,8.426,24.506,8.166,90.038
Livraria Travessa,13.471,237,16.515,10.320,13.628,9.441,10.940,74.550
Martins Fontes,9.350,2.819,11.557,11.579,8.063,4.737,9.826,57.930


## Maiores Clientes (somente os Acertos de Consignação) - (Receita Líquida)

In [8]:
a = df.loc[(df['Emissao'] > '2020') & (df['Tipo'] == 'Consignação')].groupby(
    ['Grupo', pd.Grouper(key='Emissao', freq='QS')])['Receita Líquida'].sum().sort_values(ascending=False).reset_index()

a = a.pivot_table(index='Grupo', columns='Emissao',values=
                  'Receita Líquida', aggfunc=sum, fill_value=0)

a['Total'] = a.sum(axis=1)

a = a.sort_values(by='Total', ascending=False).head(50).applymap(numero)

b = ['{:d} - Q{:d}'.format(x.year, x.quarter) for x in a.columns[:-1]]
b.append(a.columns[-1])
a.columns = b

a.head(50)

,2020 - Q1,2020 - Q2,2020 - Q3,2020 - Q4,2021 - Q1,2021 - Q2,2021 - Q3,Total
Grupo,,,,,,,,
Livraria Travessa,12.835,237,15.976,9.764,13.628,7.869,10.132,70.440
Boa Viagem Distribuidora de Livros Ltda.,3.838,2.139,1.528,3.436,16.177,19.757,21.204,68.078
Martins Fontes,9.350,1.920,7.888,8.538,6.684,4.575,9.826,48.781
Distribuidora Curitiba de Papeis e livros S/A,11.567,6.572,4.272,3.850,8.227,4.984,6.442,45.913
Saraiva e Siciliano,9.268,2.711,27.745,1.366,742,3.440,281,45.554
Catavento Distribuidora de Livros S/A,866,1.666,8.947,6.201,9.099,5.725,7.113,39.617
Itiban,3.086,2.804,4.500,5.285,4.291,11.666,4.243,35.874
Disal - Distribuidores Associados de Livros S.A,2.297,30,5.887,7.788,7.397,6.439,4.955,34.792
G.D.A. Lorandi ME,2.087,1.189,5.935,6.784,7.020,7.481,3.402,33.899


## Maiores Clientes (somente as Vendas)

In [9]:
a = df.copy()

filtro = ((a['CNPJ'].str.len() == 11) & (a['CNPJ'] != '01462931855')) | (
    a['Grupo'] == 'Eventos Veneta')

a.loc[filtro, 'Grupo'] = 'Pessoas Físicas'

a.loc[a['Grupo'].isin(COMPRAS_PUBLICAS), 'Grupo'] = 'Vendas Públicas'

filtro =  a['Grupo'].isin(COMPRAS_PUBLICAS)

a = a.loc[(a['Emissao'] > '2020') & (a['Tipo'] == 'Venda')].groupby(
    ['Grupo', pd.Grouper(key='Emissao', freq='QS')])['Receita Líquida'].sum().sort_values(ascending=False).reset_index()

a = a.pivot_table(index='Grupo', columns='Emissao',values=
                  'Receita Líquida', aggfunc=sum, fill_value=0)

a['Total'] = a.sum(axis=1)

a = a.sort_values(by='Total', ascending=False).head(50).applymap(numero)

b = ['{:d} - Q{:d}'.format(x.year, x.quarter) for x in a.columns[:-1]]
b.append(a.columns[-1])
a.columns = b

a.head(50)


,2020 - Q1,2020 - Q2,2020 - Q3,2020 - Q4,2021 - Q1,2021 - Q2,2021 - Q3,Total
Grupo,,,,,,,,
Amazon,141.741,322.429,651.686,1.142.687,112.213,313.870,365.658,3.050.282
Pessoas Físicas,118.556,25.716,68.929,331.736,46.505,242.254,44.102,877.797
Vendas Públicas,0,0,0,504.676,218.457,0,0,723.133
Inovação Distribuidora de Livros Ltda,0,1.596,2.739,47.264,23.446,54.079,19.264,148.387
Catavento Distribuidora de Livros S/A,0,299,2.846,46.300,6.566,11.038,10.924,77.974
A Página Distribuidora de Livros Ltda,0,755,21.272,6.617,6.190,18.289,6.301,59.424
MegaLeitores,0,0,0,0,0,50.119,0,50.119
Comix,3.223,424,4.213,9.349,9.563,10.091,13.074,49.937
Distribuidora Loyola de livros Ltda,-12.472,5.094,387,0,511,54.607,0,48.127


## Maiores Clientes (somente as Pessoa Física)

In [10]:
a = df.loc[(df['Emissao'] > '2020') & (df['Tipo'] == 'Venda') &
           (df['CNPJ'].str.len() == 11) & (df['CNPJ'] != '01462931855')].groupby(
    ['Grupo', pd.Grouper(key='Emissao', freq='QS')])['Receita Líquida'].sum().sort_values(ascending=False).reset_index()

a = a.pivot_table(index='Grupo', columns='Emissao',
                  values='Receita Líquida', aggfunc=sum, fill_value=0)

a['Total'] = a.sum(axis=1)

a = a.sort_values(by='Total', ascending=False).head(50).applymap(numero)

b = ['{:d} - Q{:d}'.format(x.year, x.quarter) for x in a.columns[:-1]]
b.append(a.columns[-1])
a.columns = b

a.head(50)


,2020 - Q1,2020 - Q2,2020 - Q3,2020 - Q4,2021 - Q1,2021 - Q2,2021 - Q3,Total
Grupo,,,,,,,,
Joao Carlos Pires Pinheiro,1.922,0,0,0,0,744,0,2.666
Luisa Helena Passos Rosa Scherer,0,0,0,1.871,0,0,0,1.871
MARCIO PEREIRA,0,0,150,1.211,344,0,0,1.705
NIVEA ANDRADE,0,0,0,0,0,0,1.677,1.677
ARI AGOSTINHO DA SILVA,0,0,72,1.122,0,473,0,1.666
Jucelino Neco de souza Junior,0,232,0,1.213,205,0,0,1.650
REBECCA OLIVEIRA CERQUEIRA SOUZA,0,0,0,1.617,0,0,0,1.617
ANDRE VISOCKIS,0,0,0,205,0,981,224,1.409
Marcus Vinicius Eiffle Duarte,0,173,110,0,244,637,224,1.388


## Vendas por Tipo de Cliente

In [11]:
# Lê cadastro de distribuidores
PATH = 'C:/Users/cpcle/OneDrive/Documentos/Celso/Veneta/Política Comercial/'
grp = pd.read_excel(PATH + 'relacao_clientes_nome_distribuidores.xlsx',
                    sheet_name='relacao_clientes_nome', usecols='A,C', skiprows=4, header=0, dtype=str).dropna()

tp = grp[['CNPJ']].copy()
tp['Categ'] = 'Distribuidores'

# Lê cadastro de Pontos de Venda
grp = pd.read_excel(PATH + 'relacao_clientes_nome_pontos_de_venda.xlsx',
                    sheet_name='relacao_clientes_nome_pontos_de', usecols='A,C', skiprows=4, header=0, dtype=str).dropna()
grp['Categ'] = 'Pontos de Venda'

tp = tp.append(grp[['CNPJ', 'Categ']], ignore_index=True)
del grp

# Elimina duplicidada da A PAGINA - Alocada em Distribuidores
tp = tp.loc[~((tp['CNPJ'] == '01795809000110') &
              (tp['Categ'] == 'Pontos de Venda'))]



In [12]:

# Prepara movimentos
a = df.loc[(df['Emissao'] > '2020')]
a = a.merge(tp, how='left', on=['CNPJ'], validate='many_to_one')

del tp

filtro = a['Categ'].isnull() & ((a['CNPJ'].str.len() == 11) &
                                (a['CNPJ'] != '01462931855')) 

a.loc[filtro, 'Categ'] = 'Pessoas Físicas'

a.loc[a['Grupo'].isin(COMPRAS_PUBLICAS), 'Grupo'] = 'Vendas Públicas'

a.loc[a['Grupo'].isin(GRANDES_LIVRARIAS), 'Categ'] = 'Grandes Livrarias'

filtro = a['Grupo'] == 'Eventos Veneta'
a.loc[filtro, 'Categ'] = 'Pessoas Físicas'

filtro = a['Categ'].isnull()
a.loc[filtro, 'Categ'] = a.loc[filtro, 'Grupo']

# Salva dados para Relatório em Excel
a.to_csv('../data/maiores_clientes.csv')
a = a.drop(columns=['Grupo']).rename({'Categ': 'Grupo'}, axis=1)

b = a.groupby(['Grupo', pd.Grouper(key='Emissao', freq='QS')])[
    'Receita Líquida'].sum().sort_values(ascending=False).reset_index()

a = a.groupby(
    ['Grupo', 'Tipo'], as_index=False).sum()

a = a.pivot_table(index=['Grupo'], columns=['Tipo'],
                  values=['Receita Líquida'], fill_value=0)

a[('Receita Líquida', 'Total')] = a.sum(axis=1)

a = a['Receita Líquida'][['Venda', 'Consignação', 'Outro', 'Total']
                         ].sort_values(ascending=False, by=['Total'])

a.head(50).applymap(numero)


Tipo,Venda,Consignação,Outro,Total
Grupo,,,,
Amazon,3.050.282,0,0,3.050.282
Pessoas Físicas,877.797,0,0,877.797
Vendas Públicas,723.133,0,0,723.133
Distribuidores,368.067,259.720,0,627.787
Grandes Livrarias,56.456,267.488,0,323.944
Pontos de Venda,84.104,4.141,0,88.245
MegaLeitores,50.119,0,0,50.119
Worney Almeida de Souza,28.187,0,0,28.187
CULTURA DISTRIBUIDORA DE LIVROS LTDA,3.969,9.509,0,13.478


In [13]:
a = b.pivot_table(index='Grupo', columns='Emissao',
                  values='Receita Líquida', aggfunc=sum, fill_value=0)

a['Total'] = a.sum(axis=1)

a = a.sort_values(by='Total', ascending=False).head(50).applymap(numero)

b = ['{:d} - Q{:d}'.format(x.year, x.quarter) for x in a.columns[:-1]]
b.append(a.columns[-1])
a.columns = b

a.head(50)


,2020 - Q1,2020 - Q2,2020 - Q3,2020 - Q4,2021 - Q1,2021 - Q2,2021 - Q3,Total
Grupo,,,,,,,,
Amazon,141.741,322.429,651.686,1.142.687,112.213,313.870,365.658,3.050.282
Pessoas Físicas,122.357,29.915,83.301,365.884,54.220,260.459,50.083,966.220
Vendas Públicas,0,0,0,504.676,218.457,0,0,723.133
Distribuidores,14.957,23.313,59.605,138.349,101.153,205.312,85.099,627.787
Grandes Livrarias,56.738,12.460,77.906,45.236,44.124,46.232,41.249,323.944
Pontos de Venda,5.026,6.425,13.155,13.686,15.971,15.918,18.204,88.385
MegaLeitores,0,0,0,0,0,50.119,0,50.119
Worney Almeida de Souza,5.920,3.669,2.452,2.543,4.510,5.858,3.235,28.187
CULTURA DISTRIBUIDORA DE LIVROS LTDA,2.829,0,412,486,9.521,231,0,13.478


In [14]:
#! jupyter nbconvert --to html  --no-input segmentacao_clientes.ipynb --output-dir=C:\\Users\\cpcle\\OneDrive\\Documentos\\Celso\\Python\\veneta-custo\\html
